# 1) Data Collection: Scraping Boiling Point Data from a PDF

In this notebook, we'll scrape boiling point data from a PDF copy of [The Yaws Handbook](https://www.sciencedirect.com/book/9780128008348/the-yaws-handbook-of-physical-properties-for-hydrocarbons-and-chemicals). The data tables also contain a wide variety of other chemical information. A quick inspection reveals that the tables of interest are located on pages 3-683.

*Note:* For copyright reasons, the PDF is not included in this repository.

*Note:* In this data source, the editor's confidence in the tabulated data is indicated with a code (e.g.: "Boiling Point Code"), which is a digit from 1-3. A value of 1 indicates the highest confidence in the reported value, while a value of 3 is a "rough approximation only." After collecting this data, we'll need to filter out the low-confidence data.

## Import the necessary modules.

In [1]:
import camelot # Reads PDF data tables
import pandas as pd # For basic data cleaning
from sqlalchemy import create_engine # Easy access to SQL database

## Trial PDF data table parsing with Camelot.
...before turning it loose on all 680 pages

In [3]:
trial_table = camelot.read_pdf("yaws2015.pdf", pages = "3")

In [4]:
type(trial_table)

camelot.core.TableList

Camelot returns a TableLlist object, with one entry for each table detected.

In [5]:
len(trial_table)

1

In [6]:
trial_data = trial_table[0].df
trial_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,,,,,Mol Wt,Freezing Point,,Boiling Point,,Density,,,Refractive Index,,
1,NO,FORMULA,NAME,CAS No,g/mol,"TF, K",code,"TB, K",code,"T, C",g/cm3,code,"T, C",nD,code
2,1,CAgF3O3S,silver trifluoromethanesulfonate,2923-28-6,256.937,629.15,1,---,---,---,---,---,---,---,---
3,2,CAgN,silver cyanide,506-64-9,133.884,>573.15,1,---,---,---,---,---,---,---,---
4,3,CAgNO,silver cyanate,3315-16-0,149.883,---,---,---,---,---,---,---,---,---,---
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,76,CCuNS,copper(i) thiocyanate,1111-67-7,121.630,1357.15,1,---,---,25,2.8500,1,---,---,---
78,77,CDCl3,trichloromethane-d,865-49-6,120.383,209.05,1,334.65,"1,2",20,1.5004,1,20,1.4450,1
79,78,CD4,methane-d4,558-20-3,20.067,89.78,1,112.15,"1,2",---,---,---,---,---,---
80,79,CD4O,methanol-d4,811-98-3,36.066,108.15,1,338.15,"1,2",---,---,---,---,---,---


This is pretty good! We just need to clean up the column labels a bit. It looks like the first two rows are going to be detected as data rather than labels.

## Helper functions
These functions will streamline 2 key processes:
* Scrape the data table from one PDF page using Camelot. Return results as a dataframe.
* Clean & write the data: drop unnecessary rows/columns, handle missing values, etc.), then write it to an SQL database

For the sake of both memory management & preventing data loss during unforeseen failures, we'll read one page at a time, extracting the data and saving it to the SQL database before moving on to the next page.

In [7]:
def get_table(page_num_as_str):
    
    # Get the crude data table.
    scraped_table_list = camelot.read_pdf("yaws2015.pdf", pages = page_num_as_str)
    
    # There should be exactly 1 table detected per page. If not, print an error
    # and move on using only the first table found.
    if len(scraped_table_list) != 1:
        print("Possible error reading page: ", page_num_as_str)
        
    return scraped_table_list[0].df

In [8]:
# Specify the column & row numbers to drop from each raw dataframe.
# These were identified using the exploratory scraping above.
drop_cols = [0, 4, 9, 10, 11, 12, 13, 14]
drop_rows = [0, 1]

# Specify more informative column headers.
col_labels = ["Formula", "Name", "CAS No", "Melt Pt (K)", "Melt Pt (code)", "Boil Pt (K)", "Boil Pt (code)"]

# Specify which columns should be forced to numeric data.
numeric_cols = ["Melt Pt (K)", "Melt Pt (code)", "Boil Pt (K)", "Boil Pt (code)"]

# Create a database connection via SQLAlchemy.
db_engine = create_engine("sqlite:///physical_properties.db")

def clean_and_write(raw_dataframe):
    
    # Drop unwanted rows and columns.
    clean_dataframe = raw_dataframe.drop(drop_cols, axis = 1)
    clean_dataframe = clean_dataframe.drop(drop_rows, axis = 0)
    
    # Rename the remaining columns.
    clean_dataframe.columns = col_labels
    
    # Force selected columns to numeric data. Incompatible entries such as
    # ">100" or the missing value indicator "---" are converted to NaN.
    for col in numeric_cols:
        clean_dataframe[col] = pd.to_numeric(clean_dataframe[col], errors = "coerce")
        
    # Write the data to the SQLite database.
    clean_dataframe.to_sql("yaws_2015", con = db_engine, if_exists = "append")
    
    return

## Collect the data!
Using the helper functions above, scrape the data of interest from the PDF file.

In [9]:
for page_num in range(3, 684, 1):
    
    # Cast the page number as a string, ultimately as required by Camelot.
    page_num = str(page_num)
    
    # Scrape the data, then clean & write to the database.
    raw_data = get_table(page_num)
    clean_and_write(raw_data)
    
    # Print a friendly message.
    print("Completed page {}.".format(page_num), end = "\r")
    
print("Tally-ho. All done with this batch.")

Tally-ho. All done with this batch.
